In [1]:
from datetime import datetime, timedelta
import dateutil.parser
import time
from os import walk
import pandas as pd
import re

import tushare as ts
import asyncio
from tqdm.asyncio import tqdm

In [2]:
pro = ts.pro_api('e15be55df8e1570bf0b7bd6f56c50111d19109fdad5abcb05091579b')

stock_info_list = pd.read_csv('stock_list.csv')
stock_info_list = stock_info_list.query('market == "主板"')  # Focus only on mainboard, cover almost all three-level sectors except 11 minor

stock_price = pd.read_feather('stock_price_post_adjusted.ftr')
stock_price = stock_price.merge(stock_info_list['ts_code'])

stock_industry_list = pd.read_csv('index_member_list.csv').rename(columns={'con_code': 'ts_code'})
stock_industry_list = stock_industry_list.merge(stock_info_list[['ts_code', 'list_date']])
stock_industry_list = stock_industry_list.merge(pd.read_csv('industry_list.csv'))

In [3]:
display(stock_info_list)
display(stock_price)
display(stock_industry_list)

,ts_code,symbol,name,area,industry,market,list_date,exchange,chairman,manager,secretary,reg_capital,setup_date,province,city,website,email,employees
0,000001.SZ,1,平安银行,深圳,银行,主板,19910403,SZSE,谢永林,胡跃飞,周强,1.940592e+06,19871222,广东,深圳市,www.bank.pingan.com,PAB_db@pingan.com.cn,34204.0
1,000002.SZ,2,万科A,深圳,全国地产,主板,19910129,SZSE,郁亮,祝九胜,朱旭,1.161773e+06,19840530,广东,深圳市,www.vanke.com,ir@vanke.com;peterluk@vanke.com,133455.0
2,000004.SZ,4,国华网安,深圳,互联网,主板,19910114,SZSE,黄翔,黄翔,黄冰夏,1.650526e+04,19860505,广东,深圳市,www.sz000004.cn,gnkjsz@163.com;jyz6187@qq.com;xuliruan@hotmail...,251.0
3,000005.SZ,5,世纪星源,深圳,环境保护,主板,19901210,SZSE,丁芃,郑列列,罗晓春,1.058537e+05,19890508,广东,深圳市,www.fountain.com.cn,fountain@sfc.com.cn;xiaochun@sfc.com.cn,680.0
4,000006.SZ,6,深振业A,深圳,区域地产,主板,19920427,SZSE,赵宏伟,李伟,杜汛,1.349995e+05,19890525,广东,深圳市,www.zhenye.com,szzygp@126.com,385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3929,605377.SH,605377,华旺科技,浙江,造纸,主板,20201228,SSE,张延成,张延成,黄亚芬,2.038667e+04,20091230,浙江,杭州市,www.hwpaper.cn,securities@huawon.cn,744.0
3930,605388.SH,605388,均瑶健康,湖北,软饮料,主板,20200818,SSE,王均豪,许彪,郭沁,4.300000e+04,19980805,湖北,宜昌市,www.juneyaodairy.com,juneyaodairy@juneyao.com,694.0
3931,605398.SH,605398,新炬网络,上海,软件服务,主板,20210121,SSE,孙星炎,李灏江,杨俊雄,5.949821e+03,20141104,上海,上海市,www.shsnc.com,IR@shsnc.com,1117.0
3932,605399.SH,605399,晨光新材,江西,化工原料,主板,20200804,SSE,丁建峰,丁冰,梁秋鸿,1.840000e+04,20060929,江西,九江市,www.cgsilane.com,jiangxichenguang@cgsilane.com,681.0


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,1991-04-04,48.76,48.76,48.76,48.76,49.00,-0.24,-0.4898,3.00,15.000
1,000001.SZ,1991-04-05,48.52,48.52,48.52,48.52,48.76,-0.24,-0.4922,2.00,10.000
2,000001.SZ,1991-04-08,48.04,48.04,48.04,48.04,48.52,-0.48,-0.9893,2.00,10.000
3,000001.SZ,1991-04-09,47.80,47.80,47.80,47.80,48.04,-0.24,-0.4996,4.00,19.000
4,000001.SZ,1991-04-10,47.56,47.56,47.56,47.56,47.80,-0.24,-0.5021,15.00,71.000
...,...,...,...,...,...,...,...,...,...,...,...
7398989,605500.SH,2021-01-25,18.14,18.87,17.87,18.80,18.20,0.60,3.2967,46758.17,85976.985
7398990,605500.SH,2021-01-26,19.05,19.94,18.81,19.17,18.80,0.37,1.9681,85673.46,165402.492
7398991,605500.SH,2021-01-27,18.80,19.84,18.42,19.45,19.17,0.28,1.4606,67103.67,129136.159
7398992,605500.SH,2021-01-28,18.90,19.26,18.53,18.60,19.45,-0.85,-4.3702,46080.57,86915.586


,index_code,ts_code,in_date,list_date,industry_name,level,industry_code
0,801010.SI,600467.SH,20040405,20040405,农林牧渔,L1,110000
1,801010.SI,000702.SZ,19970318,19970318,农林牧渔,L1,110000
2,801010.SI,000930.SZ,20040102,19990712,农林牧渔,L1,110000
3,801010.SI,000876.SZ,19980311,19980311,农林牧渔,L1,110000
4,801010.SI,000048.SZ,19941101,19941101,农林牧渔,L1,110000
...,...,...,...,...,...,...,...
5884,851014.SI,603322.SH,20160804,20160728,通信配套服务,L3,730203
5885,851014.SI,000971.SZ,20160701,20000427,通信配套服务,L3,730203
5886,851014.SI,603220.SH,20181122,20181115,通信配套服务,L3,730203
5887,851014.SI,603602.SH,20170817,20170810,通信配套服务,L3,730203


In [4]:
start_date = '2020-01-01'
end_date = '2020-12-31'

stock_price_selected = stock_price[['ts_code', 'trade_date', 'pct_chg']].query('@start_date <= trade_date < @end_date')
trade_days = pro.trade_cal(exchange='', start_date=start_date.replace('-',''), end_date=end_date.replace('-',''))['is_open'].sum()
stock_price_selected = stock_price_selected[stock_price_selected.groupby('ts_code')['ts_code'].transform('size') > trade_days*0.8]
stock_price_selected = stock_price_selected.merge(stock_info_list[['ts_code','name']])
stock_price_selected = stock_price_selected.pivot_table(values=['pct_chg'], index='trade_date', columns='name')
stock_price_selected

pct_chg                                                           \
name         *ST东电   *ST东科  *ST中华A   *ST中商   *ST中天   *ST中孚   *ST中新    *ST中昌   
trade_date                                                                    
2020-01-02  1.5302  1.9139  1.3268  1.0792 -2.7894  5.0002 -1.7900  10.0378   
2020-01-03  3.0143  0.0000  0.4370  0.3561  0.4101  1.2986 -0.4046  10.0455   
2020-01-06 -0.9741 -1.4080  0.2171  0.7090 -0.8162 -1.2820 -5.0813   9.9686   
2020-01-07  0.4938  0.4758  1.7358  1.4084 -1.6465  1.2986  0.4280  -2.2899   
2020-01-08  0.4875  0.4744 -2.7719 -1.7360 -3.7654  0.0000 -1.7047  -9.9604   
...            ...     ...     ...     ...     ...     ...     ...      ...   
2020-12-24  4.7923 -1.6194 -3.9682 -0.4463 -0.7291  0.0000 -4.9020   5.0225   
2020-12-25  0.0000  0.4119  0.8255  0.0000  2.2048  0.7940 -2.0605   1.5929   
2020-12-28 -3.4321  1.2291 -0.4094  1.7936 -3.5971 -0.3938  0.0000  -1.1750   
2020-12-29  0.5931 -0.4045  0.0000  4.4053 -0.7455 -3.9526  1.5796  -1.1926   
2020-12-30 -2.3539 -1.2199 -0.4111  0.4218  0.7511 -4.9381 -4.1457   0.0000   

                            ...                                          \
name         *ST中珠   *ST中绒  ...    龙元建设    龙净环保    龙头股份    龙宇燃油    龙建股份   
trade_date                  ...                                           
2020-01-02  1.1425 -2.1507  ...  0.5472  1.6411  0.7903  0.4929  0.9433   
2020-01-03 -0.5650 -2.7471  ...  0.4082  1.3118 -2.4836  0.6127  0.9345   
2020-01-06  2.8411 -1.6951  ... -0.8131 -0.3984  0.9386  2.6804 -0.6180   
2020-01-07  1.1051 -0.5747  ...  0.1367  1.5001  1.7261 -1.4242  1.8643   
2020-01-08 -2.1860 -2.3120  ... -1.5008 -0.9852 -3.1329  0.0000 -0.9145   
...            ...     ...  ...     ...     ...     ...     ...     ...   
2020-12-24 -4.1670  0.9433  ... -3.0020 -2.0090 -2.8196 -2.2073 -2.2811   
2020-12-25  0.0000  0.9345  ... -0.1934  1.7084 -0.3409  0.5302  1.9449   
2020-12-28  2.1741 -0.9259  ... -1.5503 -0.6719 -0.1713 -2.1134  0.7641   
2020-12-29 -0.7093 -0.9345  ...  5.3149  0.5637  1.0289  0.1347  0.3779   
2020-12-30  0.0000 -1.8872  ... -2.0560 -0.8969 -0.8486 -0.1345  0.3777   

                                                      
name          龙江交通     龙溪股份     龙蟠科技    龙韵股份    龙马环卫  
trade_date                                            
2020-01-02  0.6577  -1.2912   1.8027  4.0881  1.0988  
2020-01-03  0.0000  -2.2238   1.5845 -2.1753  2.0834  
2020-01-06  0.0000   1.2041   1.1008  5.0032  1.0647  
2020-01-07  0.9845   5.2875  -0.0907  8.7646  2.5463  
2020-01-08 -1.6219   0.1885   0.8175 -5.2459 -2.3973  
...            ...      ...      ...     ...     ...  
2020-12-24 -1.4545  10.0521  -8.7591 -3.1623 -4.9339  
2020-12-25  1.1055   2.7594   4.6000  0.9437  0.1899  
2020-12-28 -1.0934  -2.7778  10.0191 -4.3854 -1.4530  
2020-12-29  0.3675  -0.6666  10.0104  1.8797  3.5255  
2020-12-30 -0.7353  -1.2464  -3.3491  0.0739 -2.2910  

[242 rows x 1945 columns]

In [8]:
%%time
stock_price_selected_corr = stock_price_selected['pct_chg'].corr(method='pearson').rename_axis('company')
stock_price_selected_corr

CPU times: user 1.6 s, sys: 15.5 ms, total: 1.61 s
Wall time: 1.62 s


name,*ST东电,*ST东科,*ST中华A,*ST中商,*ST中天,*ST中孚,*ST中新,*ST中昌,*ST中珠,*ST中绒,...,龙元建设,龙净环保,龙头股份,龙宇燃油,龙建股份,龙江交通,龙溪股份,龙蟠科技,龙韵股份,龙马环卫
company,,,,,,,,,,,,,,,,,,,,,
*ST东电,1.000000,0.255227,0.194598,0.200774,0.251155,0.199008,0.181370,0.080444,0.024193,0.118781,...,0.123498,0.165862,0.078668,0.203380,0.154008,0.211935,0.290681,0.154608,0.226472,0.070998
*ST东科,0.255227,1.000000,0.169634,0.202216,0.185323,0.085253,0.247156,0.051505,0.074470,0.110919,...,0.174966,0.164833,0.076141,0.193290,0.122558,0.055942,0.211012,0.227369,0.255552,0.184261
*ST中华A,0.194598,0.169634,1.000000,0.212146,0.201821,0.258789,0.283492,0.092869,0.117124,0.176152,...,0.183641,0.294335,0.010680,0.317283,0.299111,0.290080,0.203076,0.275147,0.298123,0.136508
*ST中商,0.200774,0.202216,0.212146,1.000000,0.166662,0.220949,0.268173,0.081273,0.012555,0.250397,...,0.172274,0.236153,-0.063847,0.228889,0.204270,0.229720,0.226049,0.125557,0.240701,0.201418
*ST中天,0.251155,0.185323,0.201821,0.166662,1.000000,0.152865,0.240894,0.048177,0.081413,0.135850,...,0.050940,0.142866,-0.011721,0.184318,0.161563,0.213317,0.191708,0.254144,0.155244,0.141464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙江交通,0.211935,0.055942,0.290080,0.229720,0.213317,0.219155,0.183601,0.176225,0.027579,0.150679,...,0.207184,0.315955,-0.049102,0.337585,0.467423,1.000000,0.296984,0.247444,0.372697,0.232795
龙溪股份,0.290681,0.211012,0.203076,0.226049,0.191708,0.271388,0.224409,0.188281,0.050501,0.153294,...,0.196042,0.243291,-0.204865,0.222777,0.257413,0.296984,1.000000,0.223770,0.403111,0.181286
龙蟠科技,0.154608,0.227369,0.275147,0.125557,0.254144,0.270683,0.206368,0.154914,0.189413,0.117463,...,0.250638,0.313492,0.096971,0.280819,0.354616,0.247444,0.223770,1.000000,0.352955,0.288019


In [9]:
%%time
stock_price_selected_corr = stock_price_selected['pct_chg'].corr(method='spearman').rename_axis('company')
stock_price_selected_corr

CPU times: user 44.7 s, sys: 121 ms, total: 44.8 s
Wall time: 44.9 s


name,*ST东电,*ST东科,*ST中华A,*ST中商,*ST中天,*ST中孚,*ST中新,*ST中昌,*ST中珠,*ST中绒,...,龙元建设,龙净环保,龙头股份,龙宇燃油,龙建股份,龙江交通,龙溪股份,龙蟠科技,龙韵股份,龙马环卫
company,,,,,,,,,,,,,,,,,,,,,
*ST东电,1.000000,0.268137,0.263703,0.220484,0.243423,0.218046,0.217946,0.090562,0.095839,0.111245,...,0.173200,0.216871,0.245755,0.257350,0.200025,0.290401,0.321418,0.186250,0.287427,0.075350
*ST东科,0.268137,1.000000,0.163983,0.161248,0.172541,0.102312,0.275681,0.039134,0.117100,0.074430,...,0.159732,0.186584,0.233692,0.219589,0.159563,0.133242,0.198309,0.230576,0.274522,0.186692
*ST中华A,0.263703,0.163983,1.000000,0.290006,0.218128,0.300175,0.321305,0.113653,0.221225,0.211943,...,0.198921,0.303220,0.257360,0.329121,0.315051,0.368442,0.209220,0.297940,0.323091,0.140380
*ST中商,0.220484,0.161248,0.290006,1.000000,0.180243,0.238269,0.299291,0.145477,0.084518,0.225666,...,0.112389,0.253248,0.141193,0.291502,0.245147,0.322677,0.216378,0.143524,0.238065,0.170225
*ST中天,0.243423,0.172541,0.218128,0.180243,1.000000,0.155961,0.248637,0.056042,0.119382,0.125120,...,0.046087,0.168244,0.073115,0.222539,0.160021,0.236823,0.182809,0.236811,0.144443,0.096340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙江交通,0.290401,0.133242,0.368442,0.322677,0.236823,0.325921,0.277033,0.170493,0.185156,0.227106,...,0.253889,0.421078,0.252973,0.446087,0.505361,1.000000,0.350088,0.369203,0.440065,0.222882
龙溪股份,0.321418,0.198309,0.209220,0.216378,0.182809,0.277248,0.246778,0.137985,0.069340,0.193405,...,0.216078,0.263332,0.092969,0.236008,0.207628,0.350088,1.000000,0.244378,0.386389,0.194594
龙蟠科技,0.186250,0.230576,0.297940,0.143524,0.236811,0.286191,0.199847,0.221930,0.214245,0.143781,...,0.236012,0.336489,0.252086,0.363566,0.423813,0.369203,0.244378,1.000000,0.406497,0.288836


In [12]:
corr_df = stock_price_selected_corr[stock_price_selected_corr.abs() > 0.5]
corr_df = corr_df.unstack().dropna().drop_duplicates()
corr_df = corr_df.drop(labels=[(k1, k1) for k1, k2 in corr_df.index if k1 == k2])
corr_df = pd.DataFrame(corr_df).reset_index()
corr_df.columns = ['company', 'name', 'corr']
corr_df

,company,name,corr
0,*ST中华A,*ST宏图,0.503146
1,*ST中华A,*ST海华,0.506499
2,*ST中华A,上海三毛,0.501127
3,*ST中华A,北巴传媒,0.536063
4,*ST中昌,*ST天夏,0.538610
...,...,...,...
82915,鼎信通讯,鼎胜新材,0.525097
82916,鼎信通讯,龙净环保,0.501349
82917,鼎信通讯,龙韵股份,0.575592
82918,鼎胜新材,龙宇燃油,0.509517


In [13]:
corr_df.query('company == "泰达股份" or name == "泰达股份"')

,company,name,corr
41961,华纺股份,泰达股份,0.513599
44343,南卫股份,泰达股份,0.562338
49950,四环生物,泰达股份,0.524072
60963,岳阳兴长,泰达股份,0.501101
65461,振德医疗,泰达股份,0.583166
65819,新乡化纤,泰达股份,0.577836
70710,欣龙控股,泰达股份,0.650518
71374,江南高纤,泰达股份,0.546836
71449,江苏吴中,泰达股份,0.555153
72771,泰达股份,海王生物,0.639837


In [124]:
stock_info_list.query('ts_code == "002807.SZ" or ts_code == "600908.SH"')

,ts_code,symbol,name,area,industry,market,list_date,exchange,chairman,manager,secretary,reg_capital,setup_date,province,city,website,email,employees
1239,002807.SZ,2807,江阴银行,江苏,银行,中小板,20160902,SZSE,孙伟,宋萍,卞丹娟,217200.4131,20011203,江苏,无锡市,www.jybank.com.cn,jyrcbank@sina.com;jynsyh@sina.com,1765.0
3107,600908.SH,600908,无锡银行,江苏,银行,主板,20160923,SSE,邵辉,陶畅,孟晋,184829.9705,20050621,江苏,无锡市,www.wrcb.com.cn,contact@wrcb.com.cn,1501.0


In [ ]:
df['SPX_Prices'].pct_change()
df['SPX_Prices'].corr(df['SPX_Prices'])